## Data Dump to mysql

In [12]:
import mysql.connector as mdb
from mysql.connector import Error

In [13]:
def open_database(host, db_name, username, password):
    try:
        connection = mdb.connect(host=host,
                                 database=db_name,
                                 user=username,
                                 password=password)
        if connection.is_connected():
           db_Info = connection.get_server_info()
           print("Connected to MySQL database... MySQL Server version on ",db_Info)
           cursor = connection.cursor()
           cursor.execute("select database();")
           record = cursor.fetchone()
           print ("Your connected to - ", record)
           cursor.close()
    except Error as e :
        print ("Error while connecting to MySQL", e)
    return connection

In [14]:
def close_database(connection):
    #closing database connection.
    if(connection.is_connected()):
        connection.close()
        print("MySQL connection is closed")

In [16]:
conn = open_database('localhost', 'caselaw', 'root', 'H3rnandez!')

Connected to MySQL database... MySQL Server version on  8.0.15
Your connected to -  ('caselaw',)


---
### Data convertion and insertion

In [1]:
import pandas as pd
import numpy as np

In [2]:
metadata = pd.read_csv("../../cases_iicp.csv")

In [3]:
citations = metadata[['source','target']]

In [41]:
import traceback
import sys

In [67]:
# data.to_sql(name='Acquiring_company', con=conn, if_exists = 'replace', index=False, flavor = 'mysql')

#### Citations table

In [34]:
citations_tups = [tuple(x) for x in citations.values]

In [44]:
try:  
    cursor = conn.cursor()
    for data in citations_tups:
        query = """
                INSERT INTO citations (ecli_source, ecli_target)
                VALUES (%s, %s)
                """#%data
        result  = cursor.execute(query, data)
    conn.commit()
    cursor.close()
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

#### Cases table

In [46]:
cases = metadata[['ecli','case_label','lodge_date','document_date','ruling_name']]

In [53]:
cases_tups = [tuple(x) for x in cases.values]

In [54]:
tuples_list = cases_tups
query = """INSERT INTO cases (ecli, title, case_date, issued, description)
            VALUES (%s, %s, %s, %s, %s)
        """#%data
try:  
    cursor = conn.cursor()
    for line in tuples_list:
        result  = cursor.execute(query, line)
    conn.commit()
    cursor.close()
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

#### Subjects table (catalogue)

In [55]:
#cases = metadata[['ecli','case_label','lodge_date','document_date','ruling_name']]

In [54]:
#data = 
tuples_list = [tuple(x) for x in data.values]
query = """INSERT INTO cases (ecli, title, case_date, issued, description)
            VALUES (%s, %s, %s, %s, %s)
        """#%data
#making a function
try:  
    cursor = conn.cursor()
    for line in tuples_list:
        result  = cursor.execute(query, line)
    conn.commit()
    cursor.close()
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

#### Creator table

In [55]:
#cases = metadata[['ecli','case_label','lodge_date','document_date','ruling_name']]

In [54]:
#data = 
tuples_list = [tuple(x) for x in data.values]
query = """INSERT INTO cases (ecli, title, case_date, issued, description)
            VALUES (%s, %s, %s, %s, %s)
        """#%data
#making a function
try:  
    cursor = conn.cursor()
    for line in tuples_list:
        result  = cursor.execute(query, line)
    conn.commit()
    cursor.close()
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

## Double condition

In [ ]:
try:  
    cursor = conn.cursor()
    for data in tuples:
        if "'" in data[4]:
            query = """
            INSERT INTO `transaction` (type, date,  effectuated, researcher, comment, found_registry, Acquiring_company_ac_id, Merging_company_mc_id)
            VALUES ( '%s', '%s', %s,'%s', \"%s\",'%s',%s ,%s)
            """%data
        else:
            query = """
            INSERT INTO `transaction` (type, date,  effectuated, researcher, comment, found_registry, Acquiring_company_ac_id, Merging_company_mc_id)
            VALUES ( '%s', '%s', %s,'%s', '%s','%s',%s ,%s)
            """%data
        #print(query)
        result  = cursor.execute(query)
    conn.commit()
    cursor.close()
except Error as error :
    conn.rollback()
    print("Failed to insert into MySQL table {}".format(error))
    traceback.print_exc(file=sys.stdout)

In [207]:
close_database(conn)